In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Loading the data as Dataframe

In [ ]:
train_set = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_set = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
sample = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

# Have a look, what it look like!

In [ ]:
train_set.head()

In [ ]:
test_set.head()

In [ ]:
sample.head()

In [ ]:
x = train_set["text"]
y = train_set["target"]
print(np.shape(y))


x_test = test_set["text"]

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(x)

x_index = tokenizer.word_index
sequence = tokenizer.texts_to_sequences(x)
x_padded = pad_sequences(sequence)

In [ ]:
test_sequence = tokenizer.texts_to_sequences(x_test)
x_test_padded = pad_sequences(test_sequence)

In [ ]:
print(type(x_index))
print(type(sequence))
print(type(x_padded))

In [ ]:
print(np.shape(x_padded))
print(len(x_index))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, input_length = 10),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(x_padded, y, validation_split=0.33, epochs=10)

In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.show()

In [ ]:
predictions = model.predict(x_test_padded)
predictions = predictions.flatten()
print(predictions)
print(np.shape(predictions))

In [ ]:
predictions = (predictions>=.5)

In [ ]:
predictions = predictions.astype(int)
print(predictions)

In [ ]:
print(type(predictions))

In [ ]:
output = pd.DataFrame({'id': test_set.id, 'target': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")